## Demonstration

Once you have finished implementing your autograd engine, and it passes the tests, let's see some things that even this simple version of autograd let's you do.

In [ ]:
from engine import Value
import random
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
# approximate samples from N(0, 1)
def randn():
    return sum([random.random() for _ in range(12)]) - 6

beta_t = 0.8
sigma = 0.1

x = [randn() for _ in range(40)]
y = [beta_t * x + sigma * randn() for x in x]

In [ ]:
plt.scatter(x, y)

## Linear regression redux

Using your implemented autograd library, you should be able to find the parameter beta using gradient descent by minimising the sum of squares. I've left some of this for you to fill in the gaps to make a complete gradient descent routine.

In [ ]:
beta = Value(0.0)

def cost_function(beta):
    return sum([(beta * x_i - y_i) ** 2 for (x_i, y_i) in zip(x, y)])


loss_history = []
while True:
    
    # FILL IN HERE
    loss = cost_function(beta)
    loss.backward()
    g = beta.grad
    
    # FILL IN HERE
    
    loss_history.append(loss.data)
    # test for convergence since this is a convex function? - is grad(beta) == 0? 
    if abs(beta.grad) < 1e-6:
        print('converged')
        break
    
print(beta)

In [ ]:
plt.plot(loss_history)

Now, let's have a look at how we can use this to build a (very inefficient!) neural network library. 

Here are some simple neural network building blocks, written in terms of the Value class that you have been working on in this tutorial.

In [ ]:
class Module:

    def zero_grad(self):
        for p in self.parameters():
            p.grad = 0

    def parameters(self):
        return []

class Neuron(Module):

    def __init__(self, nin, nonlin=True):
        self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
        self.b = Value(0)
        self.nonlin = nonlin

    def __call__(self, x):
        act = sum((wi*xi for wi,xi in zip(self.w, x)), self.b)
        return act.relu() if self.nonlin else act

    def parameters(self):
        return self.w + [self.b]

    def __repr__(self):
        return f"{'ReLU' if self.nonlin else 'Linear'}Neuron({len(self.w)})"

class Layer(Module):

    def __init__(self, nin, nout, **kwargs):
        self.neurons = [Neuron(nin, **kwargs) for _ in range(nout)]

    def __call__(self, x):
        out = [n(x) for n in self.neurons]
        return out[0] if len(out) == 1 else out

    def parameters(self):
        return [p for n in self.neurons for p in n.parameters()]

    def __repr__(self):
        return f"Layer of [{', '.join(str(n) for n in self.neurons)}]"

class MLP(Module):

    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1], nonlin=i!=len(nouts)-1) for i in range(len(nouts))]

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x.sigmoid()

    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

    def __repr__(self):
        return f"MLP of [{', '.join(str(layer) for layer in self.layers)}]"

and here's some silly fake data

In [ ]:

from sklearn.datasets import make_moons, make_blobs
X, Y = make_moons(n_samples=100, noise=0.1)

# visualize in 2D
plt.figure(figsize=(5,5))
plt.scatter(X[:,0], X[:,1], c=Y, s=20, cmap='jet')

In [ ]:
net = MLP(2, [16, 16, 1]) # this has to be tiny because this scalar-valued tracking is very inefficient.
net([0.3, 0.3])

losses = []
for e in range(25):
    correct = 0
    for i in range(X.shape[0]):
        x = list(X[i])
        y = Y[i]


        net.zero_grad()
        y_pred = net(x)

        # very simple mean squared error loss
        loss = (y_pred - y) ** 2
        loss.backward()

        correct += round(y_pred.data) == y
        for p in net.parameters():
            p.data = p.data - 1e-1 * p.grad
        losses.append(loss.data)
    print('finished epoch ', e, 'accuracy', correct / X.shape[0])

In [ ]:

h = 0.25
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))
Xmesh = np.c_[xx.ravel(), yy.ravel()]
inputs = [list(map(Value, xrow)) for xrow in Xmesh]
scores = list(map(net, inputs))

In [ ]:
fig = plt.figure()
Z = np.array([s.data for s in scores])
Z = Z.reshape(xx.shape)
plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral, alpha=0.8)
plt.scatter(X[:, 0], X[:, 1], c=Y, s=40, cmap=plt.cm.Spectral)
plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())

Because we can express complex functions in terms of a small number of primitives, once we have implemented these primitives we can differentiate arbitrarily complex compositions of them by applying the chain rule over and over again!

Of course, it won't always be the most efficient way to do it, and in practice a 'real' autograd library would take advantage of vector processing, like numpy. But at it's heart it's a really simple and easy idea!

## Possible extension if you finish

We aren't limited to neural networks here - anything that can be expressed as a composition of ordinary arithmetic is attackable with autograd.

Here is some code to simulate the equations of motion of a [double pendulum](https://en.wikipedia.org/wiki/Double_pendulum), a physical system which displays chaotic behaviour and has no closed form solutions, using Eulers method

In [ ]:
# dx/dt = f(x)
# the pendulum stata is the angle of the two arms, and the corresponding angular momenta
def compound_pendulum_velocity(state):
    theta_1, theta_2, p_1, p_2 = state
    theta_1_dot = 6 * ((2 * p_1 - 3 * (theta_1 - theta_2).cos() * p_2) 
                   / (16 - 9 * (theta_1 - theta_2).cos() ** 2))
    theta_2_dot = 6 * ((8 * p_2 - 3 * (theta_1 - theta_2).cos() * p_1) 
                   / (16 - 9 * (theta_1 - theta_2).cos() ** 2))
    p_1_dot = - 0.5 * (theta_1_dot * theta_2_dot * (theta_1 - theta_2).sin()
                       + 3 * 9.8 * theta_1.sin())
    p_2_dot = - 0.5 * (-theta_1_dot * theta_2_dot * (theta_1 - theta_2).sin() + 9.8 * theta_2.sin())
    return [theta_1_dot, theta_2_dot, p_1_dot, p_2_dot]

In [ ]:

def euler(s_0, f, h=1e-3, steps=500):
    history = [s_0]
    s = s_0
    for i in range(steps):
        s = [x + h * v for (x, v) in zip(s, f(s))]
        history.append(s)
    return history
    

In [ ]:
                #    theta_1,     theta_2,    p_1,        p_2
initial_conditions = [Value(0.4), Value(0.0), Value(1.0), Value(4.0)]
dt = 1e-2
traj = euler(initial_conditions,
             compound_pendulum_velocity,
             h=dt,
            steps=500)

In [ ]:
t1, t2, _, _ = zip(*traj)
# convert to numpy to visualise

t1 = np.array([x.data for x in t1])
t2 = np.array([x.data for x in t2])

x1 = np.sin(t1)
y1 = -np.cos(t1)
x2 = x1 +  np.sin(t2)
y2 = y1 + - np.cos(t2)


In [ ]:
import matplotlib.animation as animation
from matplotlib import pyplot as plt
from IPython.display import HTML

fig = plt.figure()
ax = fig.add_subplot(111, autoscale_on=False, xlim=(-2, 2), ylim=(-2, 2))
ax.set_aspect('equal')
ax.grid()

line, = ax.plot([], [], 'o-', lw=2)
time_template = 'time = %.1fs'
time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)
dt = 0.01
def init():
    line.set_data([], [])
    time_text.set_text('')
    return line, time_text


def animate(i):
    thisx = [0, x1[i], x2[i]]
    thisy = [0, y1[i], y2[i]]

    line.set_data(thisx, thisy)
    time_text.set_text(time_template % (i*dt))
    return line, time_text


ani = animation.FuncAnimation(fig, animate, range(1, len(traj)),
                              interval=dt*1000, blit=True, init_func=init)
HTML(ani.to_html5_video())

But look - because we implemented this on top of Autograd, we can differentiate through our ODE integration routine, and get the gradient of the final positions with respect to our initial conditions

In [ ]:
traj[-1][1].backward()
initial_conditions[0].grad

## some things to play with?

The gradient is a measure of first-order sensitivity to initial conditions. Can you find initial conditions where the final position is particularly sensitive to this? What does that mean about a perturbation to the system? Can you predict initial conditions that exhibit particularly stable or chaotic behaviour using your numerically calculated gradients?

The double pendulum has an unstable equilibrium point when the pendulum is upside down, where $\theta_1 = \theta_2 = \pi$. Can you use autograd to optimise the initial momentum so that the pendulum balances in this state for as long as possible?
